In [79]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from enum import Enum

class ActivationFuction(Enum):
    SIGMOID = 1
    RELU = 2,
    ELU = 3 # a = 1

class Problem(Enum):
    Classification = 1,
    Regression = 2

class NeuralNetwork:
    def __init__(self, 
                 input_size = 784, 
                 hidden_size = 128, 
                 output_size = 10, 
                 hidden_layers_count = 1, 
                 inner_activation_function = ActivationFuction.SIGMOID,
                 problem = Problem.Classification,
                 learning_rate = 0.01, 
                 num_epochs = 10, 
                 seed = 0, 
                 use_bias = True,
                 visualize = False) -> None:
        self.seed = seed
        np.random.seed(self.seed)
        self.use_bias = use_bias
        self.hidden_layers_count = hidden_layers_count
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.visualize = visualize
        
        if problem == Problem.Regression:
            self.output_size = 1
            self.input_size = 1
        else:
            self.output_size = output_size
            
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.inner_activation_function = inner_activation_function
        self.problem = problem
 
        self.layers_sizes = [self.input_size] + [self.hidden_size] * self.hidden_layers_count + [self.output_size]
        self.layers_count = len(self.layers_sizes)
        self.weights_count = self.layers_count - 1

        self.weights = [None] * self.weights_count 
        self.bias = [None] * self.weights_count 
        self.create_weights_array()

    def create_weights_array(self):
        for i in range(self.weights_count):
            if self.inner_activation_function == ActivationFuction.SIGMOID:
                self.weights[i] = np.random.uniform(-np.sqrt(6 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), np.sqrt(6 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), size=(self.layers_sizes[i], self.layers_sizes[i + 1]))
            else:
                self.weights[i] = np.random.normal(0, np.sqrt(2 / (self.layers_sizes[i] + self.layers_sizes[i + 1])), size=(self.layers_sizes[i], self.layers_sizes[i + 1]))
    
            if self.use_bias:
                self.bias[i] = np.zeros(self.layers_sizes[i + 1])
                
    def train_and_check_acc(self, train_inputs, train_results, test_inputs, test_results):
    
        for epoch in range(self.num_epochs):
            for i in range(len(train_inputs)):
                self.train(train_inputs[i], train_results[i])
                
            if(self.visualize):
                    fig = plt.figure(figsize = (20, 12)) # width x height
                    for i in range(self.weights_count):
                        ax1 = fig.add_subplot(3, 5, i+1) # row, column, position
                        sns.heatmap(self.weights[i], cmap="coolwarm", ax=ax1) # jaka paleta? moze Spectral albo magma?
                        plt.title(f'Weight Heatmap - Layers {i}-{i+1}')
                        mean = np.mean(self.weights[i])
                        std = np.std(self.weights[i])
                        print(f'Layers {i}-{i+1}: Mean={mean}, Std={std}')
                    plt.show()
                        
                
            correct = 0
            nn_results = []
            for i in range(len(test_inputs)):
                values = self.predict(test_inputs[i])
                nn_results.append(values[-1])
                
                if (self.problem == Problem.Classification):  
                   if np.argmax(values[-1]) == np.argmax(test_results[i]):
                        correct += 1
                        
            error = self.accuracy(nn_results, test_results)  
            if (self.problem == Problem.Classification):
                test_accuracy = correct / len(test_inputs)
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Test Accuracy: {test_accuracy * 100:.2f}% Loss: {error}, Correct: {correct}, All: {len(test_inputs)} ")
            else:
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Test Accuracy: {error}")
            
    def accuracy(self, results, expected_result):
        if self.problem == Problem.Classification:
            return -np.sum(expected_result * np.log(results[-1]))
        else:
            return np.average(np.power(expected_result - results, 2))

        
    def train(self, input, expected_result): # Backpropagation
        values = self.predict(input)
    
        delta = [None] * self.weights_count
        delta[-1] = values[-1] - expected_result
        
        for weightId in range(self.weights_count - 2, -1, -1):
            delta[weightId] = np.dot(delta[weightId + 1], self.weights[weightId + 1].T) * self.neuron_activation_derivative(values[weightId + 1])
            
        for weightId in range(self.weights_count):
            self.weights[weightId] -= np.outer(values[weightId], delta[weightId]) * self.learning_rate 
            if self.use_bias:
                self.bias[weightId] = self.bias[weightId] - delta[weightId] * self.learning_rate
        
    
    def neuron_activation(self, v):
        if self.inner_activation_function == ActivationFuction.SIGMOID:
            return 1 / (1 + np.exp(-v))
        elif self.inner_activation_function == ActivationFuction.RELU:
            return np.maximum(0, v)
        elif self.inner_activation_function == ActivationFuction.ELU:
            return (v > 0) * 1 + (v <= 0) * np.exp(v) 
        
    def neuron_activation_derivative(self, v):
        if self.inner_activation_function == ActivationFuction.SIGMOID:
            return v * (1 - v)
        elif self.inner_activation_function == ActivationFuction.RELU:
            return (v > 0) * 1
        elif self.inner_activation_function == ActivationFuction.ELU:
            return (v > 0) * v + (v <= 0) * (np.exp(v) - 1) 
    
    def output_activation(self, x):
        if self.problem == Problem.Classification:
            exp_x = np.exp(x - np.max(x))
            return exp_x / exp_x.sum()
        else: 
            return x
    
    def predict(self, input): #forward propagation
        values = [None] * self.layers_count
        
        values[0] = input
        
        for weightId in range(self.weights_count):
           v = np.dot(values[weightId], self.weights[weightId]) + self.bias[weightId] if self.use_bias else 0
           if weightId != self.weights_count - 1:
               v = self.neuron_activation(v)
           else:    
               v = self.output_activation(v)
               
           values[weightId + 1] = v  
        
        return values

In [69]:
import numpy as np
from mnist.loader import MNIST

# Load the MNIST dataset
mndata = MNIST("C:\\Users\\jkobo\\source\\repos\\sn1\\mnist-digit-recognition\\data")  # Replace with the path to your MNIST data
mndata.gz = True

# Load the training and testing data
train_images, train_labels = mndata.load_training()
test_images, test_labels = mndata.load_testing()

# Convert to NumPy arrays for easier manipulation
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

num_classes = 10
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

nn = NeuralNetwork(num_epochs=25, hidden_layers_count=4, hidden_size=78,  inner_activation_function = ActivationFuction.RELU)

nn.train_and_check_acc(train_images, train_labels, test_images, test_labels)

Epoch 1/25, Test Accuracy: 94.50% Loss: 148987.8099009454, Correct: 9450, All: 10000 
Epoch 2/25, Test Accuracy: 95.80% Loss: 167702.38812139485, Correct: 9580, All: 10000 
Epoch 3/25, Test Accuracy: 96.11% Loss: 186759.3588488006, Correct: 9611, All: 10000 
Epoch 4/25, Test Accuracy: 96.72% Loss: 166758.24541385032, Correct: 9672, All: 10000 
Epoch 5/25, Test Accuracy: 95.99% Loss: 217420.3514042992, Correct: 9599, All: 10000 
Epoch 6/25, Test Accuracy: 96.82% Loss: 186973.5694611423, Correct: 9682, All: 10000 
Epoch 7/25, Test Accuracy: 97.11% Loss: 172315.7261019609, Correct: 9711, All: 10000 
Epoch 8/25, Test Accuracy: 97.09% Loss: 244473.95129345375, Correct: 9709, All: 10000 
Epoch 9/25, Test Accuracy: 96.61% Loss: 226466.33684788237, Correct: 9661, All: 10000 
Epoch 10/25, Test Accuracy: 96.95% Loss: 248806.8844620203, Correct: 9695, All: 10000 
Epoch 11/25, Test Accuracy: 97.12% Loss: 251131.37381312042, Correct: 9712, All: 10000 
Epoch 12/25, Test Accuracy: 97.30% Loss: 287672

In [ ]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.simple.test.100.csv", sep=",")
train = pd.read_csv("classification/data.simple.train.100.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 2
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 2, hidden_layers_count = 4, num_epochs = 3, inner_activation_function = ActivationFuction.RELU, visualize=True)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

In [ ]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.simple.test.10000.csv", sep=",")
train = pd.read_csv("classification/data.simple.train.10000.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 2
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 2, hidden_layers_count = 4, num_epochs = 5, inner_activation_function = ActivationFuction.SIGMOID)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

In [ ]:
import numpy as np
import pandas as pd

test = pd.read_csv("classification/data.three_gauss.test.100.csv", sep=",")
train = pd.read_csv("classification/data.three_gauss.train.100.csv", sep=",")

train_vectors = np.array(train[["x", "y"]])
train_results = np.array(train["cls"] - 1)
test_vectors = np.array(test[["x", "y"]])
test_results = np.array(test["cls"] - 1)

num_classes = 3
train_results = np.eye(num_classes)[train_results]
test_results = np.eye(num_classes)[test_results]

nn = NeuralNetwork(input_size = 2, output_size = num_classes, hidden_size = 10, hidden_layers_count = 1, num_epochs = 25, learning_rate=0.0001, inner_activation_function = ActivationFuction.RELU, visualize=True)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)

In [ ]:
import numpy as np
import pandas as pd

test = pd.read_csv("regression/data.cube.test.100.csv", sep=",")
train = pd.read_csv("regression/data.cube.train.100.csv", sep=",")

train_vectors = np.array(train["x"])
train_results = np.array(train["y"])
test_vectors = np.array(test["x"])
test_results = np.array(test["y"])

nn = NeuralNetwork(hidden_size = 6, hidden_layers_count = 1, num_epochs = 20, inner_activation_function = ActivationFuction.RELU, problem = Problem.Regression)
nn.train_and_check_acc(train_vectors, train_results, test_vectors, test_results)